## 1. 读取PLY点云文件并计算球心位置

使用项目中的 `read_ply_file` 读取点云文件，`fit_sphere_center_TLS_A` 计算球心位置，然后通过手眼关系变换到工具末端坐标系。

In [1]:
from data_process.preprocess import read_ply_file
from data_process.cloud import fit_sphere_center_TLS_A
import torch
import numpy as np
import os

def transform_point_to_tool(point, hand_eye_transform):
    """将相机坐标系中的点通过手眼关系变换到工具末端坐标系"""
    # 将点转换为齐次坐标
    point_homo = np.append(point, 1.0)
    
    # 使用手眼变换矩阵进行变换
    point_tool = hand_eye_transform @ point_homo
    
    # 只保留xyz坐标
    return point_tool[:3]

# 直接使用提供的手眼变换矩阵
hand_eye_transform = np.array([
    [ 6.79867789e-01,  7.13487890e-01,  1.69454479e-01, -1.31598869e+02],
    [-6.93042707e-01,  7.00664295e-01, -1.69591722e-01, -4.56350777e+01],
    [-2.39732343e-01, -2.13924224e-03,  9.70836664e-01,  1.03533073e+02],
    [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]
])

# 设置文件路径
ply_file_path = os.path.join(os.getcwd(), "point_cloud_00000.ply")

print(f"点云文件路径: {ply_file_path}")

# 检查文件是否存在
if not os.path.exists(ply_file_path):
    print(f"❌ 点云文件不存在: {ply_file_path}")
else:
    # 读取点云数据
    point_cloud = read_ply_file(ply_file_path)
    
    if point_cloud is not None:
        print(f"✓ 成功读取点云，包含 {len(point_cloud)} 个点")
        
        # 计算球心位置
        point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float64)
        sphere_center = fit_sphere_center_TLS_A(point_cloud_tensor)
        sphere_center_np = sphere_center.detach().numpy()
        
        print(f"球心位置 (相机坐标系): [{sphere_center_np[0]:.3f}, {sphere_center_np[1]:.3f}, {sphere_center_np[2]:.3f}] mm")

        # 转换到工具末端坐标系
        sphere_center_tool = transform_point_to_tool(sphere_center_np, hand_eye_transform)
        print(f"球心位置 (工具末端坐标系): [{sphere_center_tool[0]:.3f}, {sphere_center_tool[1]:.3f}, {sphere_center_tool[2]:.3f}] mm")

    else:
        print("❌ 点云数据读取失败")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
点云文件路径: d:\桌面\实习\diff_cali\point_cloud_00000.ply
⚠️  过滤了 2992232 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00000.ply, 153496 个点
✓ 成功读取点云，包含 153496 个点
球心位置 (相机坐标系): [6.986, -0.678, 318.206] mm
球心位置 (工具末端坐标系): [-73.412, -104.917, 410.786] mm
